In [ ]:
import talib as ta
import pandas as pd
import numpy as np
from enum import Enum
import matplotlib.pyplot as plt
import seaborn as sns

# Import untrade SDK for backtesting
try:
    from untrade.client import Client
    client = Client()
    print("untrade SDK imported successfully")
except Exception as e:
    print(f"Error importing untrade SDK: {e}")

: 

In [ ]:
class TradeType(Enum):
    """
    Enumeration defining the different types of trading actions.
    
    Attributes:
        LONG: Enter a long position
        SHORT: Enter a short position
        REVERSE_LONG: Exit a short position and enter a long position
        REVERSE_SHORT: Exit a long position and enter a short position
        CLOSE: Close any open position
        HOLD: Maintain current position (no action)
    """
    LONG = "LONG"
    SHORT = "SHORT"
    REVERSE_LONG = "REVERSE_LONG"
    REVERSE_SHORT = "REVERSE_SHORT"
    CLOSE = "CLOSE"
    HOLD = "HOLD"

## untrade SDK Integration

Let's integrate the untrade SDK to run backtests with our advanced trading strategy.

In [ ]:
def prepare_data_for_sdk(file_path):
    """Load and prepare data for use with the untrade SDK"""
    try:
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        else:
            print(f"Unsupported file format: {file_path}")
            return None
        
        # Ensure we have required columns
        required_cols = ['datetime', 'open', 'high', 'low', 'close', 'volume']
        column_map = {
            'date': 'datetime', 'time': 'datetime', 'timestamp': 'datetime',
            'Date': 'datetime', 'Time': 'datetime', 'Timestamp': 'datetime',
            'Open': 'open', 'High': 'high', 'Low': 'low', 'Close': 'close', 'Volume': 'volume'
        }
        
        # Rename columns if needed
        df = df.rename(columns={col: column_map[col] for col in df.columns if col in column_map})
        
        # Check if we still have missing columns
        missing = [col for col in required_cols if col not in df.columns]
        if missing:
            print(f"Missing required columns: {missing}")
            return None
        
        # Convert datetime
        df['datetime'] = pd.to_datetime(df['datetime'])
        
        # Sort by datetime
        df = df.sort_values('datetime')
        
        # Convert price/volume columns to numeric
        for col in ['open', 'high', 'low', 'close', 'volume']:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            
        # Drop rows with NaN values in critical columns
        df = df.dropna(subset=['open', 'high', 'low', 'close'])
        
        return df
        
    except Exception as e:
        print(f"Error preparing data: {e}")
        return None

In [ ]:
def run_backtest_with_sdk(strategy, data_path, jupyter_id="test", initial_equity=10000):
    """Run a backtest using the untrade SDK"""
    # Load and prepare data
    df = prepare_data_for_sdk(data_path)
    if df is None:
        print("Failed to prepare data for backtesting")
        return None
    
    # Run strategy
    result_df = strategy.run(df, equity=initial_equity)
    
    # Convert signals for untrade SDK
    # untrade SDK expects: 1 for buy, -1 for sell, 0 for close/hold
    signal_map = {
        TradeType.LONG.value: 1,
        TradeType.SHORT.value: -1,
        TradeType.REVERSE_LONG.value: 1,  # Buy after closing short
        TradeType.REVERSE_SHORT.value: -1,  # Sell after closing long
        TradeType.CLOSE.value: 0,
        TradeType.HOLD.value: 0
    }
    
    # Create signals column
    result_df['signals'] = result_df['trade_type'].map(signal_map)
    
    # Save to CSV for untrade SDK
    output_path = "strategy_signals.csv"
    result_df[['datetime', 'open', 'high', 'low', 'close', 'volume', 'signals']].to_csv(
        output_path, index=False
    )
    
    # Run backtest with untrade SDK
    try:
        backtest_result = client.backtest(
            jupyter_id=jupyter_id,
            file_path=output_path,
            leverage=1
        )
        
        # Display summary
        print("Backtest completed successfully.")
        print(f"Total signals: {len(result_df[result_df['signals'] != 0])}")
        print(f"Buy signals: {len(result_df[result_df['signals'] == 1])}")
        print(f"Sell signals: {len(result_df[result_df['signals'] == -1])}")
        
        return backtest_result, result_df
        
    except Exception as e:
        print(f"Error running backtest with untrade SDK: {e}")
        return None

In [ ]:
def display_backtest_metrics(backtest_result):
    """Display key metrics from untrade SDK backtest results"""
    if not backtest_result:
        print("No backtest results to display")
        return
        
    # Get the final result object
    result_list = list(backtest_result)
    if not result_list:
        print("Empty backtest results")
        return
        
    final_result = result_list[-1]
    
    # Extract metrics
    metrics = [
        ('Total Return (%)', final_result.get('total_return', 0) * 100),
        ('Annual Return (%)', final_result.get('annual_return', 0) * 100),
        ('Max Drawdown (%)', final_result.get('max_drawdown', 0) * 100),
        ('Sharpe Ratio', final_result.get('sharpe_ratio', 0)),
        ('Win Rate (%)', final_result.get('win_rate', 0) * 100),
        ('Profit Factor', final_result.get('profit_factor', 0)),
        ('Avg Win (%)', final_result.get('avg_win', 0) * 100),
        ('Avg Loss (%)', final_result.get('avg_loss', 0) * 100),
        ('# of Trades', final_result.get('total_trades', 0))
    ]
    
    # Print metrics in a formatted table
    print("\n===== Backtest Performance =====\n")
    for name, value in metrics:
        print(f"{name:<20}: {value:>10.2f}")

In [ ]:
# Example: Run a complete backtest with the strategy and untrade SDK

# Initialize strategy with custom parameters
strategy = Strategy(
    rsi_period=10,
    atr_multiplier=2.5,
    risk_pct=0.75,
    ema_fast=10,
    ema_medium=25,
    ema_slow=50
)

# Set your data path and untrade jupyter ID
DATA_PATH = "your_data.csv"  # Replace with your data file path
JUPYTER_ID = "test"         # Replace with your untrade jupyter ID

# Run backtest (comment out if not ready to execute)
# backtest_result, result_df = run_backtest_with_sdk(strategy, DATA_PATH, JUPYTER_ID)
# if backtest_result:
#     display_backtest_metrics(backtest_result)

## Visualization

Let's create some visualizations to analyze the strategy performance.

In [ ]:
def visualize_strategy_performance(result_df, backtest_result=None):
    """Create visualizations of strategy performance"""
    plt.figure(figsize=(14, 8))
    
    # Plot price with signals
    plt.subplot(2, 1, 1)
    plt.plot(result_df['datetime'], result_df['close'], label='Price', color='gray', alpha=0.7)
    
    # Plot buy signals
    buy_signals = result_df[result_df['signals'] == 1]
    if not buy_signals.empty:
        plt.scatter(buy_signals['datetime'], buy_signals['close'], 
                    color='green', marker='^', s=100, label='Buy')
    
    # Plot sell signals
    sell_signals = result_df[result_df['signals'] == -1]
    if not sell_signals.empty:
        plt.scatter(sell_signals['datetime'], sell_signals['close'], 
                    color='red', marker='v', s=100, label='Sell')
    
    plt.title('Price Chart with Trading Signals')
    plt.ylabel('Price')
    plt.grid(True, alpha=0.3)
    plt.legend()
    
    # Plot equity curve if available from backtest result
    if backtest_result:
        result_list = list(backtest_result)
        if result_list and 'equity_curve' in result_list[-1]:
            equity_data = pd.DataFrame(result_list[-1]['equity_curve'])
            equity_data['date'] = pd.to_datetime(equity_data['date'])
            
            plt.subplot(2, 1, 2)
            plt.plot(equity_data['date'], equity_data['equity'], label='Portfolio Value', color='blue')
            plt.title('Equity Curve')
            plt.ylabel('Value')
            plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()